In [1]:
ls

drive/  sample_data/


In [2]:
!pip install SimpleITK

In [3]:
cd drive/MyDrive/Colab Notebooks

/content/drive/MyDrive/Colab Notebooks


In [4]:
import os
import torch
from torch import nn, optim
from torch.utils.data import DataLoader
from data import *
from Net import *
from utils import *
import matplotlib.pyplot as plt

In [5]:
# 设置CUDA设备
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using {device}")

Using cuda


In [6]:
# filepath = r'../2'
# imagepath = r'../2/imgs'
# maskpath = r'../2/masks'
# sort_nii_image(filepath, imagepath, maskpath)

In [7]:
# 设置数据路径
dataPath = r'../2'
# 创建数据集
trainDataset = ImageDataSet(dataPath)
# 创建数据加载器
trainLoader = DataLoader(trainDataset, batch_size=1, shuffle=True)

In [8]:
# 获取trainDataset中的数据数量
dataset_size = len(trainDataset)
print(f'Train dataset size: {dataset_size}')

# 获取trainLoader中的批次数量
batch_size = len(trainLoader)
print(f'Train loader batch size: {batch_size}')

Train dataset size: 10
Train loader batch size: 10


In [9]:
# 创建网络模型
net = UNet().to(device)
#net = net.half()  # 将网络转换为半精度

# 定义损失函数和优化器
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)

# 用于保存最低损失
best_loss = float('inf')


In [10]:
for i, data in enumerate(trainLoader):
  inputs, labels = data
  print(inputs.shape)
  print(labels.shape)
  break

torch.Size([1, 1, 176, 256, 224])
torch.Size([1, 1, 256, 224, 176])


In [11]:
import pdb

In [12]:
#pdb.set_trace()
# 训练网络
for epoch in range(5):  # 根据您的需求设置训练的轮数
    epoch_loss=0
    optimizer.zero_grad()  # 在开始梯度累积之前，先清零梯度
    for i, data in enumerate(trainLoader):
        inputs, labels = data
        #inputs, labels = inputs.half(), labels.half()  # 将输入和标签转换为半精度
        inputs, labels = inputs.to(device), labels.to(device)
        # 将labels中所有非零的元素都设置为1
        labels = torch.where(labels != 0, torch.tensor(1, device=device), labels)
        #print(labels)

        # 前向传播
        outputs = net(inputs)

        labels = labels.permute(0,1, 4, 2, 3)
        # 计算损失
        loss = criterion(outputs, labels)
        epoch_loss += loss.item()

        # 反向传播和优化
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        print(f'Epoch {epoch+1}/{10}, Step {i+1}/{len(trainLoader)}, Loss: {loss.item()}')

    # 计算平均损失
    avg_loss = epoch_loss / len(trainLoader)
    # 如果平均损失更低，保存模型
    if avg_loss < best_loss:
        best_loss = avg_loss
        torch.save(net.state_dict(), f'../2/params/best_model_epoch_{epoch+1}.pth')

print('Finished Training')

Epoch 1/10, Step 1/10, Loss: 0.9754847884178162
Epoch 1/10, Step 2/10, Loss: 0.9720718860626221
Epoch 1/10, Step 3/10, Loss: 0.9537492990493774
Epoch 1/10, Step 4/10, Loss: 0.9431846141815186
Epoch 1/10, Step 5/10, Loss: 0.9093695282936096
Epoch 1/10, Step 6/10, Loss: 0.9181625843048096
Epoch 1/10, Step 7/10, Loss: 0.8845670819282532
Epoch 1/10, Step 8/10, Loss: 0.8468342423439026
Epoch 1/10, Step 9/10, Loss: 0.7967864274978638
Epoch 1/10, Step 10/10, Loss: 0.8146684169769287
Epoch 2/10, Step 1/10, Loss: 0.7888980507850647
Epoch 2/10, Step 2/10, Loss: 0.7419278621673584
Epoch 2/10, Step 3/10, Loss: 0.7163454294204712
Epoch 2/10, Step 4/10, Loss: 0.7150916457176208
Epoch 2/10, Step 5/10, Loss: 0.7173213958740234
Epoch 2/10, Step 6/10, Loss: 0.7060290575027466
Epoch 2/10, Step 7/10, Loss: 0.7032982707023621
Epoch 2/10, Step 8/10, Loss: 0.6976926922798157
Epoch 2/10, Step 9/10, Loss: 0.700623631477356
Epoch 2/10, Step 10/10, Loss: 0.6943420171737671
Epoch 3/10, Step 1/10, Loss: 0.69676274